# ID3 Decision Tree Algorithm - Jiajian Liang - 13140797

# Library Import

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

#import necessary library and api from library

# Entropy Computation

In [2]:
"""
This is function to measure information by computing its entropy/chaos
"""
def compute_entropy(y):
    if len(y) < 2: #trivial case
      return 0
    freq = np.array (y.value_counts(normalize = True))
    return -(freq * np.log2(freq+1e-6)).sum() #coded accoring to the formula computing entropy


# Information Gain Computation

In [3]:
"""
This is function to compute information gain to find the best attribute to
divide the tree
"""
def compute_info_gain(samples, attr, target):
  value = samples[attr].value_counts(normalize=True)
  split_entropy = 0
  for v, fr in value.iteritems():
    index = samples[attr]==v
    sub_entropy = compute_entropy(target[index])
    split_entropy += fr * sub_entropy
  
  entropy = compute_entropy(target)
  return entropy - split_entropy

# ID3 Decision Tree Model - Implementation

In [4]:
"""
  Model
  the id3 will be represneted as a recursively defined data structure, 
  each node will contain other nodes as it s childeren
"""
class TreeNode:
  def __init__(self):
      self.children = {} #children nodes
      self.decision = None
      self.split_feat_name = None #spliting

  
  def predict(self, sample):
    if self.decision is not None:
      print("Decision:", self.decision)
      return self.decision #return its decision if already has one
    else:
      attr_val = sample[self.split_feat_name] 
      child = self.children[attr_val]
      #print("Testing ", self.split_feat_name, "->", attr_val)
      return child.predict(sample) 
      #go to its children recursively to find the answer


  def fit(self, X, y):
        if len(X) == 0:
            self.decision = "Iris-virginica"
            #make an arbitrary decision if data is empty
            return
        else: # if it is not empty
            unique_values = y.unique() #Return unique values of Series object.
            if len(unique_values) == 1:
                self.decision = unique_values[0]
                return
            else:
                info_gain_max = 0
                for a in X.keys():
                    aig = compute_info_gain(X, a, y)
                    if aig > info_gain_max:
                        info_gain_max = aig
                        self.split_feat_name = a
                        #to find out the max attrribute
                #print(f"Split by {self.split_feat_name}, IG: {info_gain_max:.2f}")
                self.children = {}
                for v in X[self.split_feat_name].unique():
                    index = X[self.split_feat_name] == v
                    self.children[v] = TreeNode()
                    #create another treenode to perform as children/subnet
                    self.children[v].fit(X[index], y[index])
  
  """
  a nice format of displaying the decision tree
  """
  def pretty_print(self, prefix=''):
      if self.split_feat_name is not None:
          for k, v in self.children.items():
              v.pretty_print(f"{prefix}:When {self.split_feat_name} is {k}")
              #v.pretty_print(f"{prefix}:{k}:")
      else:
          print(f"{prefix}:{self.decision}")  



# Training & Evaluation

In [5]:
"""
a specfic function that implement with model(Treenode), object-oriented
allows user to modify the attribute/setting accoring to the need
"""
def id3(df, attrs, test_size, randomSeed):
  
  train, test = train_test_split(df, test_size=test_size, random_state=randomSeed)
  #spliting the dataset into train and test accoring to the ratio defined by train_size

  data = train[attrs]
  target = train['Species']
  #trainning set

  testData = test[attrs]
  testVal = test['Species']
  #testing/validation

  t = TreeNode() #init dt object
  t.fit(data, target) #fill in trainning data
  t.pretty_print() #output the learned structure/tree

  #validation part
  predictionReult = []
  Val = []

  for i in range(len(testData)):
    #print("\n")
    #print(f"Test predict sample[{i}]: \n{testData.iloc[i]}\n\tTarget: {testVal.iloc[i]}")
    #print(f"Making decision ...")
    try:
      pred = t.predict(testData.iloc[i]) #make prediction based on test/prediction dataset
    except KeyError: 
      #to avoid keyerror in case the current prediction cannot find the answer
      #tree is overfitted during the training process, needed to be pruned to improve its accuracy
      #CPP is a possibe way, unable to implement here due to lack of programming skills
      print("Overfitted model: Unable to predict this current value, in row", i, "Correct Prediction should be", testVal.iloc[i])
      pred = None

    predictionReult.append(pred)
    Val.append(testVal.iloc[i])

  match = 0
  for i in range(len(predictionReult)):
    if predictionReult[i] == Val[i]:
        match += 1

  acc = round(match / len(predictionReult), 2) 
  #to get the accuracy of this current decision tree
  return acc

In [6]:
#import iris dataset and specify its column name
df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data",
                 names = ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm', 'Species'])

attrs = ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']

test_size = 0.3
accuracy_id3 = []


"""
evalation part , training the tree
"""
for randomSeed in range(20): #should be 500, training times is reduced for faster processing
  accuracy_id3.append(id3(df, attrs, test_size, randomSeed))
  
accuracy_id3= np.array(accuracy_id3)
print(f"\n\nOverall Performance by Accuracy mean: {accuracy_id3.mean():.2f}")
#overall performance/accuracy

:When PetalLengthCm is 3.5:Iris-versicolor
:When PetalLengthCm is 5.5:Iris-virginica
:When PetalLengthCm is 5.7:Iris-virginica
:When PetalLengthCm is 5.0:When SepalWidthCm is 2.2:Iris-virginica
:When PetalLengthCm is 5.0:When SepalWidthCm is 2.5:Iris-virginica
:When PetalLengthCm is 5.0:When SepalWidthCm is 3.0:Iris-versicolor
:When PetalLengthCm is 5.8:Iris-virginica
:When PetalLengthCm is 3.9:Iris-versicolor
:When PetalLengthCm is 6.1:Iris-virginica
:When PetalLengthCm is 4.7:Iris-versicolor
:When PetalLengthCm is 3.8:Iris-versicolor
:When PetalLengthCm is 4.9:When PetalWidthCm is 1.8:Iris-virginica
:When PetalLengthCm is 4.9:When PetalWidthCm is 1.5:Iris-versicolor
:When PetalLengthCm is 5.1:Iris-virginica
:When PetalLengthCm is 4.5:When SepalLengthCm is 4.9:Iris-virginica
:When PetalLengthCm is 4.5:When SepalLengthCm is 6.0:Iris-versicolor
:When PetalLengthCm is 4.5:When SepalLengthCm is 6.2:Iris-versicolor
:When PetalLengthCm is 4.5:When SepalLengthCm is 5.7:Iris-versicolor
:When 